In [89]:
import os
import json
import requests
import time
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv(
    dotenv_path=".env",
    override=True
)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
  api_key= os.getenv("AZURE_OPENAI_API_KEY"),
  api_version="2024-05-01-preview"
)

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=f"You are a helpful AI assistant who makes interesting visualizations based on data." 
    f"You have access to a sandboxed environment for writing and testing code."
    f"When you are asked to create a visualization you should follow these steps:"
    f"1. Write the code."
    f"2. Anytime you write new code display a preview of the code to show your work."
    f"3. Run the code to confirm that it runs."
    f"4. If the code is successful display the visualization."
    f"5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
    tools=[{"type": "code_interpreter"}],
    model="gpt-4o" #You must replace this value with the deployment name for your model.
)

print(assistant.model_dump_json(indent=2))

{
  "id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
  "created_at": 1734736413,
  "description": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "metadata": {},
  "model": "gpt-4o",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    },
    "file_search": null
  },
  "top_p": 1.0

In [90]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_89HVhvptsm9XH7iloi8CyHJT', created_at=1734736420, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [91]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Create a visualization of a sinewave"
)

thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_EGgXZPIIhYL8jZcxnhupV136",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Create a visualization of a sinewave"
          },
          "type": "text"
        }
      ],
      "created_at": 1734736426,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_89HVhvptsm9XH7iloi8CyHJT"
    }
  ],
  "object": "list",
  "first_id": "msg_EGgXZPIIhYL8jZcxnhupV136",
  "last_id": "msg_EGgXZPIIhYL8jZcxnhupV136",
  "has_more": false
}


In [92]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

In [93]:
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print(run.model_dump_json(indent=2))
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))


Status: completed
{
  "id": "run_jvqPPaDIAC9rgl5sWd1PrR3u",
  "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
  "cancelled_at": null,
  "completed_at": 1734736440,
  "created_at": 1734736434,
  "expires_at": null,
  "failed_at": null,
  "incomplete_details": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "last_error": null,
  "max_completion_tokens": null,
  "max_prompt_tokens": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "thread.run",
  "paral

In [94]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_FRjlWC2r3Al14R4sohTZQ1ou",
      "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-gHTMJp35Bg1lWn6UVqmynllU",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's a visualization of a sine wave. The x-axis represents the values from 0 to \\(2\\pi\\), and the y-axis represents the sine of those values. The graph oscillates smoothly between -1 and 1, illustrating the periodic nature of the sine function. If you have any specific modifications or another visualizations you'd like, please let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734736439,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.m

In [95]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

assistant-gHTMJp35Bg1lWn6UVqmynllU


In [96]:
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [97]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()


Ask a follow-up question on the thread

In [98]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to generate the sinewave"
)

In [99]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

In [100]:
start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print(run.model_dump_json(indent=2))
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))


Status: completed
{
  "id": "run_E5gPDBQbb7oMR4GB0pEm5TO9",
  "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
  "cancelled_at": null,
  "completed_at": 1734736529,
  "created_at": 1734736527,
  "expires_at": null,
  "failed_at": null,
  "incomplete_details": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "last_error": null,
  "max_completion_tokens": null,
  "max_prompt_tokens": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "thread.run",
  "paral

In [101]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_JCf6cToHaBMQwgya37Egjfy6",
      "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Sure! Here is the code used to generate the sine wave visualization:\n\n```python\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Generate data for the sine wave\nx = np.linspace(0, 2 * np.pi, 1000)\ny = np.sin(x)\n\n# Create the plot\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='Sine Wave')\n\n# Add title and labels\nplt.title('Sine Wave')\nplt.xlabel('x')\nplt.ylabel('sin(x)')\nplt.legend()\n\n# Show the plot\nplt.grid(True)\nplt.show()\n```\n\nIf you have any questions or need further assistance, feel free to ask!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734736528,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "objec

In [102]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

Sure! Here is the code used to generate the sine wave visualization:

```python
import matplotlib.pyplot as plt
import numpy as np

# Generate data for the sine wave
x = np.linspace(0, 2 * np.pi, 1000)
y = np.sin(x)

# Create the plot
plt.figure(figsize=(10, 5))
plt.plot(x, y, label='Sine Wave')

# Add title and labels
plt.title('Sine Wave')
plt.xlabel('x')
plt.ylabel('sin(x)')
plt.legend()

# Show the plot
plt.grid(True)
plt.show()
```

If you have any questions or need further assistance, feel free to ask!


Dark mode

In [103]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I prefer visualizations in darkmode can you change the colors to make a darkmode version of this visualization."
)

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [104]:
start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print(run.model_dump_json(indent=2))
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
{
  "id": "run_Em9NnXtysA2vzNBg6mMLKy5J",
  "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
  "cancelled_at": null,
  "completed_at": 1734736625,
  "created_at": 1734736621,
  "expires_at": null,
  "failed_at": null,
  "incomplete_details": null,
  "instructions": "You are a helpful AI assistant who makes interesting visualizations based on data.You have access to a sandboxed environment for writing and testing code.When you are asked to create a visualization you should follow these steps:1. Write the code.2. Anytime you write new code display a preview of the code to show your work.3. Run the code to confirm that it runs.4. If the code is successful display the visualization.5. If the code is unsuccessful display the error message and try to revise the code and rerun going through the steps from above again.",
  "last_error": null,
  "max_completion_tokens": null,
  "max_prompt_tokens": null,
  "metadata": {},
  "model": "gpt-4o",
  "object": "thread.run",
  "paral

In [105]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_d8YeP0ki2cyK4porJzMFm6P3",
      "assistant_id": "asst_TBSuxWOzW9pXa8arrzPPYP7J",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-V2lg1eYUZitmQIjCaBY1mQiA",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here's the dark mode version of the sine wave visualization. The background is now dark, with the sine wave colored in cyan for better contrast. The titles, labels, and grid lines are also adjusted to be more visible on a dark background.\n\nIf there's anything else you'd like to modify or if you have any other visualization requests, let me know!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734736624,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "threa

In [106]:
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id'] # index numbers can vary if you have had a different conversation over the course of the thread.

print(image_file_id)

content = client.files.content(image_file_id)
image= content.write_to_file("dark_sine.png")

# Display the image in the default image viewer
image = Image.open("dark_sine.png")
image.show()

assistant-V2lg1eYUZitmQIjCaBY1mQiA
